In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression  # this is your ALGORITHM
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll import scope

import mlflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('taxi')

df = pd.read_pickle('../tripdata.pkl')
df['duration'] = (df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']).dt.total_seconds()/60
df.drop(columns=['VendorID', 'store_and_fwd_flag', 'RatecodeID', 'passenger_count',
                  'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tolls_amount', 
                  'improvement_surcharge', 'congestion_surcharge', 'tip_amount', 'ehail_fee', 
                  'total_amount', 'trip_type', 'lpep_pickup_datetime','lpep_dropoff_datetime'], inplace=True)

df = df[(df.duration > 1) & (df.duration < 60)]

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

df['pu_do'] = df['PULocationID'] + '_' + df['DOLocationID']

numerical = ['trip_distance']
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

train = xgb.DMatrix(X_train, label=y_train)

# define a training objective using hyperopt
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': int(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': int(params['min_child_samples']),
        'objective': 'reg:squarederror',
    }
    
    # train the model
    model = xgb.train(params, train)
    
    # make predictions
    y_pred = model.predict(train)
    
    # calculate the error
    rmse = root_mean_squared_error(y_train, y_pred)
    
    return {'loss': rmse, 'status': STATUS_OK}

space = {
    'max_depth': scope.int(hp.quniform('max_depth', 5, 15, 1)),
    'gamma': hp.uniform('gamma', 0.01, 0.5),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0.01, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'num_leaves': scope.int(hp.quniform('num_leaves', 10, 100, 1)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_child_samples': scope.int(hp.quniform('min_child_samples', 10, 100, 1)),
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)
